只有两句的对话

In [1]:
import json
 
finename = "test.txt"
f = open(finename, encoding='utf-8')
lines = f.readlines()

print(lines)

['为 什 么 听 一 面 之 辞 就 相 信 ？ 只 有 当 事 人 才 能 知 道 吧 ， 咱 们 旁 观 者 并 不 知 道 真 相 啊\t因 为 另 一 个 当 事 人 不 站 出 来 啊 … 站 出 来 就 能 比 较 了 。\n', '我 也 要 去 健 身 懒 半 年 了\t我 一 直 懒 现 在 终 于 动 起 来 了\n', '小 红 酒 不 断 啊\t我 看 家 里 有 几 箱 ， 没 事 酌 两 杯\n', '打 包 2 斤 牛 油 回 来 ， 谢 谢 。\t我 谢 谢 你 …\n']


In [2]:
lines_2 = []

for line in lines:
    lines_2.append(line.strip('\n').split('\t'))

print(lines_2)

[['为 什 么 听 一 面 之 辞 就 相 信 ？ 只 有 当 事 人 才 能 知 道 吧 ， 咱 们 旁 观 者 并 不 知 道 真 相 啊', '因 为 另 一 个 当 事 人 不 站 出 来 啊 … 站 出 来 就 能 比 较 了 。'], ['我 也 要 去 健 身 懒 半 年 了', '我 一 直 懒 现 在 终 于 动 起 来 了'], ['小 红 酒 不 断 啊', '我 看 家 里 有 几 箱 ， 没 事 酌 两 杯'], ['打 包 2 斤 牛 油 回 来 ， 谢 谢 。', '我 谢 谢 你 …']]


In [3]:
jsontext ={"XDAI-Form-ZH":{"Source":"LCCC","Topic":"","Components":{"talkers":["A","B"],"dlg":[{"talker":0,"text":""},{"talker":1,"text":""}],"qak":[{"talker":0,"text":""},{"talker":1,"text":""}],"query":{"talker":0,"text":""},"mask":{"talker":1,"text":"[sMASK]"},"Postdlg":[{"talker":1,"text":"xxxx"}]}}, "Fomats":{"intra_utter_form":"{talker}:{text}","inter_utter_token":"|","prompt_form":"背景：{bgk} 对话：{concat([dlg,qak,Query])} {talkers[1]}:[sMASK]{concat([Postdlg]})"}, "Input":"", "Output":""}

In [4]:
def fill_in(jsontext, sentence1, sentence2):
    jsontext['XDAI-Form-ZH']['Components']['dlg'][0]['text'] = sentence1
    jsontext['XDAI-Form-ZH']['Components']['query']['text'] = sentence1
    jsontext['Input'] = '对话：A：' + jsontext['XDAI-Form-ZH']['Components']['dlg'][0]['text'] + 'A：' + jsontext['XDAI-Form-ZH']['Components']['query']['text'] + 'B：' + jsontext['XDAI-Form-ZH']['Components']['mask']['text']
    jsontext['Output'] = sentence2
    return jsontext

In [5]:
with open('./dialogue_2.json', 'a') as dialogue:
        dialogue.write(str(fill_in(jsontext, lines_2[0][0], lines_2[0][1])).replace("'", '''"'''))

多于两句的对话

In [4]:
import json

path = "./dialogue_json.json"

with open(path, "r", encoding='utf-8') as f:
    dialogs = json.load(f)

dialogs = dialogs['valid']

print(dialogs)

[['遭 淋 安 逸 了 ？', '是 哈 ， 你 没 遭 撒 ？', '晚 班', '那 起 来 这 么 早 ， 这 天 气 多 适 合 睡 觉', '睡 不 着 了'], ['生 快', '姐 也 要 快 哈', '恩 恩 昨 晚 玩 的 开 心 吗 ？', '挺 嗨 的 哦']]


In [5]:
AB = ["A", "B"]

for m in range(len(dialogs)):
    
    dialog = dialogs[m]
    ques_index = 0
    
    #当对话长度>=3时执行
    if len(dialog) >=3:
        
        #找到句尾含有？的最后一句话的位置
        for sent_index in range(len(dialog)):
            if (dialog[sent_index].endswith('？')) and (sent_index >= 1) and (sent_index <= len(dialog)-2):
                ques_index = sent_index
        
        #如果这句话前后都有句子
        if (ques_index >= 1) and (ques_index <= len(dialog)-2):
            
            ##输出前面的所有句子，这句话，和后面的两句话
            #for n in range(ques_index+2):
            #    print(dialog[n])
            #print('------------')
            
            #未被填充的json
            jsontext ={"XDAI-Form-ZH":{"Source":"LCCC","Topic":"","Components":{"talkers":["A","B"],"dlg":[],"qak":[],"query":{"talker":'',"text":""},"mask":{"talker":'',"text":"[sMASK]"},"Postdlg":[]}}, "Fomats":{"intra_utter_form":"{talker}:{text}","inter_utter_token":"|","prompt_form":"背景：{bgk} 对话：{concat([dlg,qak,Query])} {talkers[1]}:[sMASK]{concat([Postdlg]})"}, "Input":"对话：", "Output":""} 
            
            #填充json的dlg部分
            for n in range(ques_index):
                jsontext["XDAI-Form-ZH"]["Components"]["dlg"].append({"talker":0})
                if n%2 == 0:
                    jsontext["XDAI-Form-ZH"]["Components"]["dlg"][n]["talker"] = 0
                else:
                    jsontext["XDAI-Form-ZH"]["Components"]["dlg"][n]["talker"] = 1
                jsontext["XDAI-Form-ZH"]["Components"]["dlg"][n]["text"] = dialog[n]
                jsontext['Input'] = jsontext['Input'] + AB[n%2] + '：' + dialog[n]
            
            #填充json的query部分
            jsontext["XDAI-Form-ZH"]["Components"]['query']['talker'] = ques_index%2
            jsontext["XDAI-Form-ZH"]["Components"]['query']['text'] = dialog[ques_index]
            jsontext['Input'] = jsontext['Input'] + AB[ques_index%2] + '：' + dialog[ques_index]
            
            
            #填充json的mask部分
            jsontext['Input'] = jsontext['Input'] + AB[(ques_index+1)%2] + '：' + '[sMASK]'
            jsontext['Output'] = dialog[ques_index+1]
            jsontext["XDAI-Form-ZH"]["Components"]['mask']["talker"]=(ques_index+1)%2
            
            #填充json的Postdlg部分
            if ques_index+2 < len(dialog):
                for n in range((len(dialog)-1) - (ques_index+2) + 1):
                    jsontext["XDAI-Form-ZH"]["Components"]['Postdlg'].append({"talker":0})
                    jsontext["XDAI-Form-ZH"]["Components"]['Postdlg'][n]["talker"] = (ques_index+1+n+1)%2
                    jsontext["XDAI-Form-ZH"]["Components"]['Postdlg'][n]["text"] = dialog[ques_index+1+n+1]
                    jsontext['Input'] = jsontext['Input'] + AB[(ques_index+1+n+1)%2] + '：' + dialog[ques_index+1+n+1]
            
            #输出json
            with open('./{}.json'.format(m), 'a') as dialogue:
                dialogue.write(str(jsontext).replace("'", '''"'''))